# SQLMemory
- python의 Database ORM인 sqlalchemy가 지원하는 모든 데이터베이스에 채팅 기록을 저장할 수 있는 `SQLChatMessageHistory` 클래스를 이용한다.

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()
print(os.environ["MODEL_ID"])

meta-llama/Meta-Llama-3-8B-Instruct


# 사용법
- `SQLChatMessageHistory`를 사용하려면 `session_id`와 `connection_string`이 필요하다. `sesseion_id`는 세션의 고유 ID이고, `connection_string`은 데이터베이스에 연결을 지정하는 문자열이다.
- `add_user_message`로 사용자 메세지를 기록
- `add_ai_message`로 AI 메세지 기록

In [3]:
from langchain_community.chat_message_histories import SQLChatMessageHistory

# SQLChatMessageHistory 객체를 생성하고 세션 ID와 데이터베이스 연결 문자열을 전달합니다.
chat_message_history = SQLChatMessageHistory(
    session_id="sql_chat_history", connection_string="sqlite:///sqlite.db"
)

# 사용자 메시지를 추가합니다.
# chat_message_history.add_user_message(
#     "Hi! My name is Teddy. I am a AI programmer. Nice to meet you!"
# )
# AI 메시지를 추가합니다.
# chat_message_history.add_ai_message("Hi Teddy! Nice to meet you too!")

In [5]:
print(chat_message_history.messages)

[]


In [6]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
)
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_models.huggingface import ChatHuggingFace
from langchain_community.llms import HuggingFaceEndpoint

llm = HuggingFaceEndpoint(
    repo_id=os.environ["MODEL_ID"], 
    max_new_tokens=1024,
    temperature=0.1,
    huggingfacehub_api_token=os.environ["HF_API_KEY"],
)
model = ChatHuggingFace(llm=llm)

/home/dudaji/anaconda3/envs/llm-study/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceEndpoint` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the from langchain-huggingface package and should be used instead. To use it run `pip install -U from langchain-huggingface` and import as `from from langchain_huggingface import llms import HuggingFaceEndpoint`.
  warn_deprecated(
/home/dudaji/anaconda3/envs/llm-study/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/dudaji/.cache/huggingface/token
Login successful


/home/dudaji/anaconda3/envs/llm-study/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatHuggingFace` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the from langchain-huggingface package and should be used instead. To use it run `pip install -U from langchain-huggingface` and import as `from from langchain_huggingface.chat_models import huggingface import ChatHuggingFace`.
  warn_deprecated(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


`LCEL Runnables`를 사용하면 chain과 쉽게 결합할 수 있다.

In [7]:
prompt = ChatPromptTemplate.from_messages(
    [
        # 시스템 메시지를 설정하여 어시스턴트의 역할을 정의합니다.
        ("system", "You are a helpful assistant."),
        # 이전 대화 내용을 포함하기 위한 플레이스홀더를 추가합니다.
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),  # 사용자의 질문을 입력받는 메시지를 설정합니다.
    ]
)
chain = prompt | model

# RunnableWithMessageHistory
- `RunnableWithMessageHistory`는 chain에 message history를 추가할 수 있게 하는 클래스이다.  
- 항상 message history factory에 적합한 매개변수를 config에 포함시켜야 한다.
- 두 번째 인자는 `BaseChatMessageHistory`를 반환하는 함수가 들어가야 한다.
- 자세한 내용은 [문서](https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html) 참고

In [8]:
chain_with_history = RunnableWithMessageHistory(
    chain,
    lambda session_id: SQLChatMessageHistory(
        session_id=session_id, connection_string="sqlite:///sqlite.db"
    ),  # session_id를 기반으로 SQLChatMessageHistory 객체를 생성하는 람다 함수
    input_messages_key="question",  # 입력 메시지의 키를 "question"으로 설정
    history_messages_key="history",  # 대화 기록 메시지의 키를 "history"로 설정
)

In [9]:
config = {"configurable": {"session_id": "sql_chat_history"}}

In [10]:
chain_with_history.invoke({"question": "Hi my name is nopro"}, config=config)

AIMessage(content="Nice to meet you, Nopro! It's great to have you here. Is there something I can help you with or would you like to chat about something in particular? I'm all ears!", id='run-8b1e4d65-c1c7-4b9a-a524-2cefa0e203cf-0')

In [11]:
chain_with_history.invoke({"question": "Whats my name?"}, config=config)

AIMessage(content='Your name is Nopro!', id='run-fdc994c9-0833-4a92-bc22-8a356a0e8bb1-0')